## 数据解析

正则 bs4 xpath pyquery

图片爬取

requests
urllib和requests的作用一样，但是当requests一问世就迅速取代了urllib


In [ ]:
import requests
import urllib
headers={
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

In [ ]:
## requests
url='https://bkimg.cdn.bcebos.com/pic/ac6eddc451da81cb2f9a02a25d66d01609243164?x-bce-process=image/watermark,image_d2F0ZXIvYmFpa2U5Mg==,g_7,xp_5,yp_5/format,f_auto'
response=requests.get(url=url,headers=headers)
image_data=response.content
with open('./data/pic1.jpeg','wb') as fp:
    fp.write(image_data)

In [ ]:
## urlib
urllib.request.urlretrieve(url,'./data/pic1_cp.jpeg')

使用urllib无法进行UA伪装，而requests可以

爬取校花网中的数据

Elements中包含了完整页面的源码数据，包括动态加载的数据；而network只包含了某个请求的数据（不包含动态加载）

结论：爬虫时一定要进行页面布局分析，如果当前页面没有动态加载数据，则可以直接使用Elements进行解析，否则只能使用network进行解析

In [ ]:
##要加上re.S回车和换行
import re
import os
dirName='./data/imagelib'
if not os.path.exists(dirName):
    os.mkdir(dirName)
url='http://www.521609.com/qingchunmeinv'
page_text=requests.get(url=url,headers=headers).text
ex='<li>.*?<img src="(.*?)" width=.*?</li>'
image_src_list = re.findall(ex,page_text,re.S)
for src in image_src_list:
    src = 'http://www.521609.com' + src
    image_path = dirName + '/' + src.split('/')[-1]
    urllib.request.urlretrieve(src,image_path)
    print(src+"下载成功！")

数据解析是用来实现聚焦爬虫

### bs4

BeautifulSoup(fp,'lxml')
BeautifulSoup(page_text,'lxml')

In [43]:
page_text='<html><body><title>数据解析</title></body></html>'

In [ ]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(page_text,'lxml')
## soup.tagName只返回第一次出现的满足要求的
## soup.find(tagName, attribute_name=''):属性定位
## soup.findAll():跟find一样，但是返回所有的
soup.body
soup.find('body')

In [ ]:
## select选择器
## >表示一个层级
## 空格表示多个层级
soup.select('body > title')

.text返回该标签下所有文本内容 .string返回该标签下直系文本内容

tag['attribute']取属性

三国演义

In [ ]:
url='https://www.shicimingju.com/book/sanguoyanyi.html'
page_text_response = requests.get(url=url,headers=headers)
page_text_response.encoding='utf-8'
page_text = page_text_response.text
soup = BeautifulSoup(page_text,'lxml')

In [ ]:
a_list = soup.select('.book-mulu > ul > li > a')

In [ ]:
a_list

In [ ]:
fp = open('./data/sanguoyanyi.txt','w',encoding='utf-8')
for a in a_list:
    title = a.string
    detail_url = 'http://www.shicimingju.com'+a['href']
    response = requests.get(url=detail_url,headers=headers)
    response.encoding='utf-8'
    page_text_detail = response.text
    soup = BeautifulSoup(page_text_detail,'lxml')
    div_tag = soup.find('div',class_='chapter_content')
    content = div_tag.text
    fp.write(title + ':' + content + '\n')
    print(title, '保存成功！')
fp.close()

### xpath

etree.parse('filename'):将本地html加载到该队相中 etree.HTML(page_text)

标签定位：  
最左侧/表示该xpath表达式一定从根标签开始定位标签，即一定从html开始；  
非最左侧的/表示一个层级；最左侧的//表示可以从任意标签开始定位；  
属性定位tagName[@attribute='value']，接着后面可以继续根/  
索引定位：//p[2]，下标从1开始  
模糊匹配：  
  //div[contains(@class,"ng")]  
  //div[starts-with(@class,"ta")]

取文本  
/text()  直系文本内容  
//text() 所有文本内容

取属性  
/@attrName  

In [45]:
from lxml import etree
tree = etree.HTML(page_text)

In [48]:
tree.xpath('/html/body/title')
tree.xpath('/html//title')
tree.xpath('//title')

[<Element title at 0x111905d08>]

在局部数据解析中，xpath要使用./操作，./表示的就是当前的局部数据

In [60]:
dirName = './data/nature_pic'
import os 
if not os.path.exists(dirName):
    os.mkdir(dirName)
url='http://pic.netbian.com/4kfengjing/'
response = requests.get(url=url,headers=headers)
response.encoding='gbk'
page_text = response.text
# print(page_text)
tree = etree.HTML(page_text)
li_list = tree.xpath('//div[@class="slist"]/ul/li')
print(len(li_list))

for li in li_list:
    title = li.xpath('./a/img/@alt')[0] + '.jpg'
    img_src = 'http://pic.netbian.com' + li.xpath('./a/img/@src')[0]
    print(img_src)
    image_data = requests.get(url=img_src,headers=headers).content
    with open(dirName + '/'+title,'wb') as fp:
        fp.write(image_data)

20
http://pic.netbian.com/uploads/allimg/180826/113958-1535254798fc1c.jpg
http://pic.netbian.com/uploads/allimg/210628/220732-1624889252aba1.jpg
http://pic.netbian.com/uploads/allimg/210419/203429-1618835669e1dd.jpg
http://pic.netbian.com/uploads/allimg/210810/231712-16286086323788.jpg
http://pic.netbian.com/uploads/allimg/180315/110404-152108304476cb.jpg
http://pic.netbian.com/uploads/allimg/170725/103840-15009503208823.jpg
http://pic.netbian.com/uploads/allimg/170609/123945-14969831856c4d.jpg
http://pic.netbian.com/uploads/allimg/210826/000055-16299072552135.jpg
http://pic.netbian.com/uploads/allimg/210812/230911-162878095180f6.jpg
http://pic.netbian.com/uploads/allimg/210822/220228-16296409484d2c.jpg
http://pic.netbian.com/uploads/allimg/210811/221726-1628691446d92b.jpg
http://pic.netbian.com/uploads/allimg/210628/221136-1624889496686d.jpg
http://pic.netbian.com/uploads/allimg/210717/003431-16264532710e32.jpg
http://pic.netbian.com/uploads/allimg/210725/233656-16272274161bd5.jpg
htt

bs4直接解析出携带html信息文本串  
xpath表达式更具有通用性  
xpath可以使用管道符号表示管道符号左右两侧同时生效，如果只有一个生效，那就一个生效

图片懒加载  
    需要爬取伪属性，requests本身是没有可视化范围的
学过的反爬机制  
   robots  
    UA伪装  
    动态加载  
    图片懒加载